In [2]:
import numpy as np

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [5]:
torch.cuda.is_available()

True

In [9]:
class Net(nn.Module):

    def __init__(self):
        # this is the place where you instantiate all your modules
        # you can later access them using the same names you've given them in
        # here
        super(Net, self).__init__()
        
        
    def setup_model(
        self, 
        domain_dims,
        emd_dim,
        domain_dims,
        domain_emb_wt,
        lstm_dim,
        interaction_layer_dim,
        intermediate_context_dims,
        
    ):
        self.num_domains = len(domain_dims)
        self.domain_dims = domain_dims
        self.list_Entity_Embed = []
        self.interaction_layer_dim = interaction_layer_dim
        for d in domain_dims:
            e = nn.Embedding(num_embeddings=d, embedding_dim=emd_dim)
            self.list_Entity_Embed.append(e)
        
        
        self.BD_LSTM_layer = nn.LSTM(
            input_size = emd_dim,
            hidden_size = lstm_dim,
            num_layers = 1,
            batch_first = True,
            bidirectional = True
            )
        
        self.FNN_1 = nn.Linear(self.lstm_dim*2, intermediate_context_dims[0])
        self.FNN_2 = nn.Linear(intermediate_context_dims[0], self.interaction_layer_dim) 
           
        return
    
    
    # Define network structure
    def forward(self, input_x_pos, input_x_neg=None):
        
        def process(x):
            x = torch.chunk(x, chunks = self.num_domains)
            for i in range(self.num_domains) :
                pass
                
            return x
        
        
        return x

SyntaxError: duplicate argument 'domain_dims' in function definition (<ipython-input-9-87bea1ae4bae>, line 13)

In [10]:
net = Net()
print(net)

NameError: name 'Net' is not defined